# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [288]:
# import libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sqlalchemy import create_engine

from gensim.test.utils import datapath
from gensim import utils
import gensim.models

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss, zero_one_loss, jaccard_score, accuracy_score, auc
import joblib

#Glove
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#Doc2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
import multiprocessing

[nltk_data] Downloading package punkt to /home/tarciso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tarciso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tarciso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
messages_df = pd.read_sql_table(con=engine, table_name='Message')
categories_df = pd.read_sql_table(con=engine, table_name='MessageCategoryWide')

In [98]:
# prepare data for ML Pipeline
X = messages_df.message.values
Y_df = categories_df.drop('message_id', axis=1)
Y = Y_df.values
category_columns = Y_df.columns

In [99]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [100]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [101]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [102]:
# create categories_list data with a list of the categories for each message
categories = pd.read_csv('../data/disaster_categories.csv')
categories['categories_list'] = categories['categories'].apply(lambda x: re.sub(r"[_a-z]+-0[;]?", "", x)) \
                                    .apply(lambda x: re.sub(r"-1", "", x)) \
                                    .apply(lambda x: re.sub(r";$", "", x))

In [103]:
categories

,id,categories,categories_list
0,2,related-1;request-0;offer-0;aid_related-0;medi...,related
1,7,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;other_aid;weather_related;...
2,8,related-1;request-0;offer-0;aid_related-0;medi...,related
3,9,related-1;request-1;offer-0;aid_related-1;medi...,related;request;aid_related;medical_products;o...
4,12,related-1;request-0;offer-0;aid_related-0;medi...,related
...,...,...,...
26243,30261,related-0;request-0;offer-0;aid_related-0;medi...,
26244,30262,related-0;request-0;offer-0;aid_related-0;medi...,
26245,30263,related-1;request-0;offer-0;aid_related-0;medi...,related
26246,30264,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;military


### 2 - Design and Apply Tokenization Strategies

#### 2.1 - Write down tokenization functions

In [110]:
def tokenize_to_list(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A list of tokens extracted from the input text
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tokens

In [112]:
def tokenize_to_str(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A string with the tokens extracted from the input text concatenated by spaces
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]
    #Return tokens list as a string joined by whitespaces
    clean_tokens_str = ' '.join(clean_tokens)

    return clean_tokens_str

In [113]:
lemmatizer = WordNetLemmatizer()
tokenize_to_list(X[0],lemmatizer)

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [114]:
tokenize_to_str(X[0],lemmatizer)

'weather update cold front cuba could pas haiti'

#### 2.2 - Generate tokenized messages table for further analysis

In [116]:
%%time
lemmatizer = WordNetLemmatizer()
messages_df['tokens_list'] = messages_df.message.apply(lambda x: tokenize_to_list(x, lemmatizer))

CPU times: user 1min 26s, sys: 7.19 s, total: 1min 33s
Wall time: 1min 33s


In [117]:
%%time
messages_df['tokens_str'] = messages_df.message.apply(lambda x: tokenize_to_str(x, lemmatizer))

CPU times: user 1min 32s, sys: 7.56 s, total: 1min 40s
Wall time: 1min 40s


In [118]:
messages_df

,message_id,message,original,genre,num_words,tokens,tokens_list,tokens_str
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13,weather update cold front cuba could pas haiti,"[weather, update, cold, front, cuba, could, pa...",weather update cold front cuba could pas haiti
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9,hurricane,[hurricane],hurricane
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6,looking someone name,"[looking, someone, name]",looking someone name
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13,un report leogane 80 90 destroyed hospital st ...,"[un, report, leogane, 80, 90, destroyed, hospi...",un report leogane 80 90 destroyed hospital st ...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12,say west side haiti rest country today tonight,"[say, west, side, haiti, rest, country, today,...",say west side haiti rest country today tonight
...,...,...,...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,21,training demonstrated enhance micronutrient pr...,"[training, demonstrated, enhance, micronutrien...",training demonstrated enhance micronutrient pr...
26140,30262,A suitable candidate has been selected and OCH...,None,news,22,suitable candidate selected ocha jakarta curre...,"[suitable, candidate, selected, ocha, jakarta,...",suitable candidate selected ocha jakarta curre...
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,23,proshika operating cox bazar municipality 5 un...,"[proshika, operating, cox, bazar, municipality...",proshika operating cox bazar municipality 5 un...
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,31,2 000 woman protesting conduct election tearga...,"[2, 000, woman, protesting, conduct, election,...",2 000 woman protesting conduct election tearga...


In [121]:
messages_tokens = messages_df[['message_id','tokens_str']]

In [122]:
messages_tokens.to_sql('MessageTokens', engine, index=False, if_exists='replace')

In [127]:
#messages_categories = messages_tokens.merge(categories_df, on='message_id')

In [128]:
#messages_categories.head()

#### 2.3 - Find n-grams and save to DB

In [133]:
def get_ngrams_freqs(messages_array, n=1):
    vec = CountVectorizer(ngram_range=(n, n)).fit(messages_array)
    bag_of_words = vec.transform(messages_array)
    word_count = bag_of_words.sum(axis=0)
    words_freq = [(word, n, word_count[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
    words_freq_df = pd.DataFrame(data = words_freq, columns = ['ngram','n','count'])
    return words_freq_df

In [135]:
unigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str)
bigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=2)
trigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=3)
ngrams_freqs = pd.concat([unigrams_freqs, bigrams_freqs, trigrams_freqs])

In [136]:
ngrams_freqs

,ngram,n,count
0,water,1,3034
1,people,1,2998
2,food,1,2892
3,help,1,2649
4,need,1,2484
...,...,...,...
289130,crisis identifying crisis,3,1
289131,identifying crisis function,3,1
289132,crisis function hiv,3,1
289133,function hiv aid,3,1


In [137]:
ngrams_freqs.to_sql('NGramsFreqs', engine, index=False, if_exists='replace')

### 3 - Split dataset
- Split data into train and test sets
- Train pipeline

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=199)

In [139]:
X_train

array(['crudess you are the most charm and charismatic DT member .. thanks for all your videos in Santiago.. and to talk with me twice',
       'Tidal waves triggered by an earthquake off Indonesia on Sunday swept over a vast swath of coastlines, including those in India, Indonesia, Malaysia, the Maldives, Somalia, Sri Lanka and Thailand.',
       'The earthquake happened at 4h34 in the evening and for the night. ',
       ...,
       'Earthquake of the day #Haiti or Google possibly leaving China ?',
       'When the matter had been debated previously, the Non-Aligned Movement foreign ministers firmly stated that there was no right to humanitarian intervention.',
       '@DumboNYC : Oh no RT @endtwist : ConEd power station in #DUMBO is flooding ! #nopower #sandy'],
      dtype=object)

In [140]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [141]:
print(X_train.shape, y_train.shape)

(19608,) (19608, 36)


___

## Feature Engineering

### 4 - Create a Mean/TF-IDF word vector from a locally trained Word2Vec model

#### 4.1 - Train a Word2Vec model using messages text

In [156]:
class Message(object):
    """An interator that yields messages tokens (lists of str)."""
    
    def __init__(self, messages_df, sample_size=-1):
        self.messages_df = messages_df
        self.sample_size = sample_size

    def __iter__(self):
        messages = messages_df
        
        #Sample dataset if specified
        if self.sample_size > 0:
            messages = messages.sample(self.sample_size)
        
        #Yeld tokenized message joined by whitespaces
        for token_str in messages.tokens_str:
            yield token_str.split()

In [157]:
messages_df.tokens_str

0           weather update cold front cuba could pas haiti
1                                                hurricane
2                                     looking someone name
3        un report leogane 80 90 destroyed hospital st ...
4           say west side haiti rest country today tonight
                               ...                        
26139    training demonstrated enhance micronutrient pr...
26140    suitable candidate selected ocha jakarta curre...
26141    proshika operating cox bazar municipality 5 un...
26142    2 000 woman protesting conduct election tearga...
26143    radical shift thinking came result meeting rec...
Name: tokens_str, Length: 26144, dtype: object

In [158]:
%%time

sentences = Message(messages_df=messages_df)
local_w2v_model_full = gensim.models.Word2Vec(sentences=sentences)
local_w2v_model_full.save("messages-word2vec-full.model")

CPU times: user 6.85 s, sys: 62.3 ms, total: 6.91 s
Wall time: 3.95 s


In [159]:
w2v_model_full = gensim.models.Word2Vec.load("messages-word2vec-full.model")

In [160]:
# look up generated word vocab
for i, word in enumerate(w2v_model_full.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [161]:
# look up generated word vectors
w2v_model_full.wv['earthquake']

array([ 0.21104944, -0.3319796 , -1.089296  ,  0.18319671, -1.3858151 ,
        0.41997495, -0.44799736, -0.31913823,  0.26479647,  0.22596444,
       -0.67827404,  0.6387983 , -0.32434493,  0.22279064,  0.6625759 ,
       -0.60558397,  0.7741924 , -0.4315002 ,  0.21657471,  0.14816046,
        0.2847021 , -0.26252496,  0.07353719,  0.7520718 , -1.1380043 ,
       -0.45561278,  1.2611561 ,  0.6042986 , -0.791835  ,  0.7173501 ,
        0.23810938,  0.08721794, -0.7173073 ,  0.1417906 , -1.235521  ,
       -0.20525241,  0.8515887 , -0.7559804 , -0.12912439, -0.20614992,
        0.46374416, -0.11519865,  0.08970786, -0.6659644 , -1.2046064 ,
       -0.1547625 , -0.6765996 , -0.8617822 ,  0.52614105,  0.20374697,
       -0.20562942,  0.44847372,  0.4006036 , -0.3305939 , -0.890827  ,
       -0.18864353, -0.1102459 , -0.7055123 ,  0.6534887 , -0.20928772,
       -0.6761006 ,  0.16414587,  0.6912797 , -1.1202699 , -0.31452778,
       -0.25494555, -0.48690298,  0.9590672 ,  0.12348403,  0.85

#### 4.2 - Implement Vector Aggregator

Inspecting variables

In [34]:
len(X)

26144

In [35]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [36]:
X.shape[0]

26144

Initial implementation of aggregators

In [163]:
from collections import defaultdict

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [166]:
#Testing defaultdict functionality
test_dict = defaultdict(lambda: 1, [("Hello" , 7), ("hi" , 10), ("there" , 45),("at" , 23),("this" , 77)])
test_dict['Hi']

1

In [167]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

Test aggregators using a Pipeline

In [171]:
%%time

w2v_mean_pip = Pipeline([
    ('features',FeatureUnion([
        ('w2v_mean', MeanEmbeddingVectorizer(w2v_model_full)),
        ('w2v_tfidf', TfidfEmbeddingTrainVectorizer(w2v_model_full))
    ])),    
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWar

              precision    recall  f1-score   support

   related-0       0.42      0.09      0.14      1532
   related-1       0.77      0.92      0.84      4950

   micro avg       0.75      0.72      0.74      6482
   macro avg       0.60      0.50      0.49      6482
weighted avg       0.69      0.72      0.67      6482

              precision    recall  f1-score   support

   request-0       0.83      0.94      0.88      5393
   request-1       0.21      0.08      0.11      1143

    accuracy                           0.79      6536
   macro avg       0.52      0.51      0.50      6536
weighted avg       0.72      0.79      0.74      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.15      0.26      6506
     offer-1       0.01      0.97      0.01        30

    accuracy                           0.16      6536
   macro avg       0.50      0.56      0.14      6536
weighted avg       0.99      0.16      0.26      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  refugees-0       0.98      0.11      0.19      6322
  refugees-1       0.03      0.94      0.07       214

    accuracy                           0.14      6536
   macro avg       0.51      0.52      0.13      6536
weighted avg       0.95      0.14      0.19      6536

              precision    recall  f1-score   support

     death-0       0.97      0.74      0.84      6244
     death-1       0.08      0.52      0.14       292

    accuracy                           0.73      6536
   macro avg       0.53      0.63      0.49      6536
weighted avg       0.93      0.73      0.81      6536

              precision    recall  f1-score   support

 other_aid-0       0.86      0.96      0.91      5660
 other_aid-1       0.10      0.03      0.05       876

    accuracy                           0.83      6536
   macro avg       0.48      0.49      0.48      6536
weighted avg       0.76      0.83      0.79      6536

                   

#### 4.3 - Create Estimator to both train and average w2v on corpus data

Trying W2VTransformer - Gensim's sklearn Transformer

In [177]:
from gensim.sklearn_api import W2VTransformer
from gensim.test.utils import common_texts

model = W2VTransformer(size=10, min_count=1, seed=1)

model.fit(common_texts)
          
model.transform(['graph', 'system'])

array([[-0.03371398,  0.02059478,  0.03959151, -0.02205233, -0.04617335,
        -0.01969593, -0.00457998, -0.00582213,  0.02125652, -0.01535213],
       [ 0.02142361, -0.00582616, -0.02330563, -0.0109981 ,  0.04116236,
         0.00383347, -0.04030757, -0.01917825, -0.00613602,  0.00246344]],
      dtype=float32)

In [178]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [179]:
messages_df.head(5).tokens.str.split().values

array([list(['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']),
       list(['hurricane']), list(['looking', 'someone', 'name']),
       list(['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']),
       list(['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'])],
      dtype=object)

In [180]:
messages_df.head(5).tokens

0       weather update cold front cuba could pas haiti
1                                            hurricane
2                                 looking someone name
3    un report leogane 80 90 destroyed hospital st ...
4       say west side haiti rest country today tonight
Name: tokens, dtype: object

In [181]:
w2v_model = W2VTransformer()
w2v_model.fit(messages_df.tokens.str.split().values)

W2VTransformer(alpha=0.025, batch_words=10000, cbow_mean=1,
               hashfxn=<built-in function hash>, hs=0, iter=5,
               max_vocab_size=None, min_alpha=0.0001, min_count=5, negative=5,
               null_word=0, sample=0.001, seed=1, sg=0, size=100,
               sorted_vocab=1, trim_rule=None, window=5, workers=3)

In [182]:
word = messages_df.tokens[0].split()[1]
print(word)
w2v_model.transform(word)

update


array([[ 0.03542764, -0.28579733, -0.3234617 , -0.17000724, -0.10690939,
         0.36051524,  0.10642287, -0.2585234 , -0.13454503, -0.01546795,
        -0.3532414 ,  0.11675725,  0.02408516,  0.02258891,  0.11721277,
        -0.36799595,  0.08041113, -0.13952191, -0.11258205, -0.04382391,
         0.03592162, -0.07439654,  0.1974777 ,  0.08637668,  0.01859285,
        -0.37983087,  0.12240552,  0.03861552, -0.20716593,  0.25008574,
         0.23566355, -0.12843259, -0.23219466,  0.25452614, -0.2211008 ,
        -0.27766627, -0.02740289,  0.2483707 ,  0.11817191, -0.12598895,
         0.25037935,  0.12809387, -0.02996706, -0.13629319, -0.4203338 ,
        -0.190851  ,  0.01853267,  0.08945538, -0.07898644,  0.38430724,
        -0.09168626,  0.22904907,  0.09345771,  0.2171086 , -0.18064429,
         0.00498869, -0.10208338, -0.3013055 ,  0.14297897,  0.05437218,
        -0.17450488,  0.01276202,  0.06973235, -0.36280915, -0.12941052,
        -0.27365053, -0.03801227,  0.37044048, -0.1

In [183]:
test_model = gensim.models.Word2Vec(messages_df.tokens.str.split().values)

In [184]:
for i, word in enumerate(test_model.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [185]:
test_model.wv['earthquake']

array([ 0.37413156, -0.29447675, -1.1473513 ,  0.1399712 , -1.3577124 ,
        0.38972723, -0.5248779 , -0.27431366,  0.18564814,  0.32011175,
       -0.92048323,  0.7638822 , -0.3467795 ,  0.20631939,  0.69229835,
       -0.5137698 ,  0.78849727, -0.53712076,  0.3273621 ,  0.08093651,
        0.1796478 , -0.18000315,  0.03477437,  0.5881102 , -1.0303501 ,
       -0.57792044,  1.2664272 ,  0.70872444, -0.6136005 ,  1.0286759 ,
        0.03119387,  0.13206565, -0.87440485,  0.24148501, -1.3372976 ,
       -0.03095893,  0.8894171 , -0.48485777, -0.19779691, -0.31983602,
        0.39436466, -0.2030403 , -0.19268915, -0.6927799 , -0.76853496,
       -0.11196443, -0.5107505 , -0.84635997,  0.6195139 ,  0.40368935,
       -0.28698164,  0.42348206,  0.4112247 , -0.18532372, -0.97496283,
       -0.23311763, -0.17038721, -0.6933485 ,  0.8033593 , -0.13250948,
       -0.5810229 ,  0.03634665,  0.5506473 , -0.9300046 , -0.20617561,
       -0.28819388, -0.5183132 ,  0.7458735 ,  0.0432611 ,  0.86

Re-implement aggregators now with training capability

In [186]:
class MeanEmbeddingTrainVectorizer(object):

    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.vector_size
            
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [187]:
class TfidfEmbeddingTrainVectorizer(object):
    
    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
        
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.vector_size
            
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

Test aggregators using a Pipeline

In [189]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('features',FeatureUnion([
        ('w2v_mean', MeanEmbeddingTrainVectorizer(w2v_model_full)),
        ('w2v_mean_train', MeanEmbeddingTrainVectorizer()),
        ('w2v_tfidf', TfidfEmbeddingTrainVectorizer(w2v_model_full)),
        ('w2v_tfidf_train', TfidfEmbeddingTrainVectorizer()),
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWar

              precision    recall  f1-score   support

   related-0       0.40      0.04      0.07      1532
   related-1       0.77      0.94      0.84      4950

   micro avg       0.76      0.72      0.74      6482
   macro avg       0.58      0.49      0.46      6482
weighted avg       0.68      0.72      0.66      6482

              precision    recall  f1-score   support

   request-0       0.83      0.95      0.88      5393
   request-1       0.24      0.08      0.12      1143

    accuracy                           0.79      6536
   macro avg       0.53      0.51      0.50      6536
weighted avg       0.73      0.79      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.16      0.28      6506
     offer-1       0.01      0.97      0.01        30

    accuracy                           0.16      6536
   macro avg       0.50      0.56      0.14      6536
weighted avg       0.99      0.16      0.27      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  refugees-0       0.98      0.12      0.21      6322
  refugees-1       0.03      0.94      0.07       214

    accuracy                           0.15      6536
   macro avg       0.51      0.53      0.14      6536
weighted avg       0.95      0.15      0.21      6536

              precision    recall  f1-score   support

     death-0       0.99      0.13      0.23      6244
     death-1       0.05      0.96      0.09       292

    accuracy                           0.17      6536
   macro avg       0.52      0.54      0.16      6536
weighted avg       0.94      0.17      0.22      6536

              precision    recall  f1-score   support

 other_aid-0       0.90      0.22      0.35      5660
 other_aid-1       0.14      0.84      0.24       876

    accuracy                           0.30      6536
   macro avg       0.52      0.53      0.30      6536
weighted avg       0.80      0.30      0.34      6536

                   

### 5. Use Glove pre-trained model to generate feature vectors

Glove pre-trained vectors can be downloaded here: https://nlp.stanford.edu/projects/glove/

In [54]:
# Code which can be used to convert from Glove default format to Gensim W2V format

glove_file = 'glove-pretrained/glove.6B.50d.txt'
tmp_file = 'glove-pretrained/glove.6B.50d_word2vec.txt'

#_ = glove2word2vec(glove_file, tmp_file)

glove_model = KeyedVectors.load_word2vec_format(tmp_file)

In [56]:
from gensim.models import KeyedVectors

glove_300d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.300d_word2vec.txt', binary=False)

##### Use a Pipeline to train the Word2Vec estimator

Test Aggregators with Glove using a Pipeline

In [290]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('features',FeatureUnion([
        ('glove_mean', MeanEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ])),    
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())
print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

   related-0       0.42      0.08      0.13      1532
   related-1       0.77      0.94      0.85      4950

   micro avg       0.75      0.74      0.74      6482
   macro avg       0.59      0.51      0.49      6482
weighted avg       0.69      0.74      0.68      6482

              precision    recall  f1-score   support

   request-0       0.83      0.95      0.89      5393
   request-1       0.27      0.09      0.13      1143

    accuracy                           0.80      6536
   macro avg       0.55      0.52      0.51      6536
weighted avg       0.73      0.80      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.99      0.99      6506
     offer-1       0.00      0.00      0.00        30

    accuracy                           0.99      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.99      0.99      0.99      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

infrastructure_related-0       0.97      0.20      0.34      6111
infrastructure_related-1       0.07      0.90      0.14       425

                accuracy                           0.25      6536
               macro avg       0.52      0.55      0.24      6536
            weighted avg       0.91      0.25      0.32      6536

              precision    recall  f1-score   support

 transport-0       0.98      0.20      0.33      6219
 transport-1       0.05      0.91      0.10       317

    accuracy                           0.23      6536
   macro avg       0.52      0.55      0.22      6536
weighted avg       0.93      0.23      0.32      6536

              precision    recall  f1-score   support

 buildings-0       0.98      0.20      0.33      6209
 buildings-1       0.06      0.92      0.11       327

    accuracy                           0.23      6536
   macro avg       0.52      0.56      0.22      6536
we

In [292]:
w2v_mean_pip.decision_function(X_test)

AttributeError: 'MultiOutputClassifier' object has no attribute 'decision_function'

In [314]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(len(category_columns)):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_pred[:, i])
    #average_precision[i] = average_precision_score(y_test[:, i], y_pred[:, i])

# A "micro-average": quantifying score on all classes jointly
# precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
#     y_score.ravel())
# average_precision["micro"] = average_precision_score(Y_test, y_score,
#                                                      average="micro")
# print('Average precision score, micro-averaged over all classes: {0:0.2f}'
#       .format(average_precision["micro"]))

ValueError: multiclass format is not supported

In [318]:
from sklearn.metrics import hamming_loss, zero_one_loss, jaccard_score, accuracy_score, auc, recall_score, precision_score

print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Recall = ", recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Precision = ", precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))

Hamming Loss =  0.026774785801713587
Zero-One Loss =  0.026774785801713552
Jaccard Score =  0.947846818655938
Accuracy =  0.9732252141982864
Recall =  0.9732252141982864
Precision =  0.9732252141982864


In [347]:
y_test[0] == y_pred[0]

array([ True,  True,  True,  True,  True,  True, False, False, False,
        True,  True,  True,  True, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True])

In [349]:
y_test[0]

array([1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [350]:
y_pred[0]

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [341]:
6536*36

235296

In [329]:
( == (np.argmax(y_pred, axis=1) == 1)

array([ True,  True,  True, ...,  True,  True,  True])

In [276]:
print(np.argmax(y_pred, axis=1).shape)
print(y_pred.shape)

(6536,)
(6536, 36)


In [248]:
from sklearn.preprocessing import MultiLabelBinarizer
train_foo = [['sci-fi', 'thriller'],['comedy'],['sci-fi', 'thriller'],['comedy']]
mlb = MultiLabelBinarizer()
mlb_label_train = mlb.fit_transform(train_foo)
mlb_label_train

array([[0, 1, 1],
       [1, 0, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [249]:
y_test

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [250]:
y_pred

array([[1, 0, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ...,
       [1, 0, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 1, 1, 0]])

In [261]:
Y_df[Y_df['related'] == 2]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
458,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20264,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20435,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22260,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23316,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Use Doc2Vec to generate feature vectors

In [192]:
# Checking the number of words in the corpus
messages_df['message'].apply(lambda x: len(x.split(' '))).sum()

631288

Try Doc2Vec step by step

In [193]:
def read_messages(messages, tokens_only=False):
    for index, tokens_str in np.ndenumerate(messages):
        tokens = tokens_str.split()
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            tags = index
            yield gensim.models.doc2vec.TaggedDocument(tokens, tags)

train_corpus = list(read_messages(X_train))
test_corpus = list(read_messages(X_test, tokens_only=True))

In [196]:
train_corpus[:10]

[TaggedDocument(words=['crudess', 'you', 'are', 'the', 'most', 'charm', 'and', 'charismatic', 'DT', 'member', '..', 'thanks', 'for', 'all', 'your', 'videos', 'in', 'Santiago..', 'and', 'to', 'talk', 'with', 'me', 'twice'], tags=(0,)),
 TaggedDocument(words=['Tidal', 'waves', 'triggered', 'by', 'an', 'earthquake', 'off', 'Indonesia', 'on', 'Sunday', 'swept', 'over', 'a', 'vast', 'swath', 'of', 'coastlines,', 'including', 'those', 'in', 'India,', 'Indonesia,', 'Malaysia,', 'the', 'Maldives,', 'Somalia,', 'Sri', 'Lanka', 'and', 'Thailand.'], tags=(1,)),
 TaggedDocument(words=['The', 'earthquake', 'happened', 'at', '4h34', 'in', 'the', 'evening', 'and', 'for', 'the', 'night.'], tags=(2,)),
 TaggedDocument(words=['Experience', ':', 'soup', 'kitchen', 'coordinator', '.', 'Preparation', 'of', 'care', 'packages', 'and', 'toilety', 'kits', '.', 'Handing', 'out', 'food', 'and', 'supplies', '.', 'Billingual', '(', 'English/Spanish', ')'], tags=(3,)),
 TaggedDocument(words=['what', 'is', 'the', 'b

In [197]:
test_corpus[:10]

[['7',
  'Coys',
  'of',
  'CRPF',
  'also',
  'comprising',
  'RAF',
  'PERSONNEL',
  'alongwith',
  'relief',
  'materiaal',
  'and',
  '2',
  'doctors',
  'alongwith',
  'medicines,',
  'health-kit',
  'reached',
  'Gujarat.'],
 ['can',
  'of',
  'olives',
  ',',
  'can',
  'of',
  'chickpeas',
  ',',
  'bag',
  'of',
  'pasta',
  ',',
  'bag',
  'of',
  'hard',
  'pretzels',
  ',',
  '(',
  'also',
  'have',
  'sneakers',
  'I',
  'can',
  'donate',
  ')'],
 ['We',
  'are',
  'at',
  'the',
  'top',
  'of',
  'Fontamara',
  '43',
  '(area',
  'of',
  'Carrefour),',
  'at',
  'the',
  'top',
  'of',
  'the',
  'hill',
  'named',
  'label..',
  'unrecognized',
  'characters.',
  'We',
  'did',
  'not',
  'get',
  'any',
  'assistance:',
  'water,',
  'food,',
  'tents,',
  'medicine.'],
 ['The',
  'premier',
  'stressed',
  'that',
  'it',
  'has',
  'been',
  'a',
  'tradition',
  'for',
  'the',
  'Chinese',
  'nation',
  'and',
  'a',
  'socialistic',
  'advantage',
  'to',
  'sup

In [198]:
d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=5, epochs=10)

In [199]:
d2v_model.build_vocab(train_corpus)

In [200]:
%%time 

d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

CPU times: user 52.6 s, sys: 3.68 s, total: 56.3 s
Wall time: 40.7 s


#### Create a Doc2Vec Estimator/Transformer 

In [202]:
class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, epochs=20):
        self.epochs = epochs
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1
        self.d2v_model = None

    def fit(self, X, y):
        tagged_x = [TaggedDocument(tokens_str.split(), [index]) for index, tokens_str in np.ndenumerate(X)]
        self.d2v_model = Doc2Vec(vector_size=self.vector_size, workers=self.workers, epochs=self.epochs)
        self.d2v_model.build_vocab(tagged_x)
        self.d2v_model.train(tagged_x, total_examples=self.d2v_model.corpus_count, 
                             epochs=self.epochs)

        return self

    def transform(self, X):
        return np.asmatrix(np.array([self.d2v_model.infer_vector(tokens_str.split())
                                     for tokens_str in X]))

In [203]:
%%time

doc2vec_trf = Doc2VecTransformer(epochs=1)
doc2vec_trf.fit(X_train, y_train)

CPU times: user 14.7 s, sys: 1.47 s, total: 16.2 s
Wall time: 13.4 s


Doc2VecTransformer(epochs=1, vector_size=100)

In [204]:
X_test[0].split()

['7',
 'Coys',
 'of',
 'CRPF',
 'also',
 'comprising',
 'RAF',
 'PERSONNEL',
 'alongwith',
 'relief',
 'materiaal',
 'and',
 '2',
 'doctors',
 'alongwith',
 'medicines,',
 'health-kit',
 'reached',
 'Gujarat.']

In [207]:
inferred_vector = doc2vec_trf.d2v_model.infer_vector(X_test[0].split())
print(inferred_vector.shape)
print(type(inferred_vector))
print(inferred_vector)

inferred_vector2 = np.asmatrix(np.array([doc2vec_trf.d2v_model.infer_vector(X_test[0].split())]))
print(inferred_vector2.shape)
print(type(inferred_vector2))
print(inferred_vector2)

(100,)
<class 'numpy.ndarray'>
[ 6.31834287e-03  4.61347256e-04 -1.29872141e-02 -4.80164401e-03
  3.59207485e-03  8.66516773e-03  2.32114596e-03  9.10063682e-05
  4.23087645e-03  7.88450148e-03 -1.36282656e-03 -1.24735385e-03
 -7.98782217e-04 -8.08076747e-03 -2.45363713e-04  5.05756121e-04
  3.70248314e-03 -3.39294737e-03  9.95638035e-03 -5.60440635e-03
  9.83770005e-03 -1.03835016e-02 -1.44825699e-02  3.55671812e-03
 -7.08816294e-03 -9.07991175e-03  1.33448010e-02  9.98355262e-03
  2.76246364e-03  4.64056525e-03  1.50831370e-03 -3.35493730e-03
 -5.74394828e-04  1.06374444e-02  1.74646999e-03 -2.96263443e-03
  3.74071859e-03  2.22042960e-04  2.82107200e-03  3.26577411e-03
  7.82914273e-03  5.42608835e-03 -5.87662682e-03  2.58797663e-03
  2.35243067e-02 -3.32232309e-03  4.31253575e-03 -1.08276894e-02
  4.10266861e-04  6.95886265e-04  2.74938531e-03 -1.31242943e-03
  4.62129060e-03  3.00153950e-03 -4.45416151e-03  3.17062368e-03
  2.96346983e-03  1.25447882e-03 -7.45772291e-03  6.7223520

In [208]:
X_train

array(['crudess you are the most charm and charismatic DT member .. thanks for all your videos in Santiago.. and to talk with me twice',
       'Tidal waves triggered by an earthquake off Indonesia on Sunday swept over a vast swath of coastlines, including those in India, Indonesia, Malaysia, the Maldives, Somalia, Sri Lanka and Thailand.',
       'The earthquake happened at 4h34 in the evening and for the night. ',
       ...,
       'Earthquake of the day #Haiti or Google possibly leaving China ?',
       'When the matter had been debated previously, the Non-Aligned Movement foreign ministers firmly stated that there was no right to humanitarian intervention.',
       '@DumboNYC : Oh no RT @endtwist : ConEd power station in #DUMBO is flooding ! #nopower #sandy'],
      dtype=object)

In [209]:
X_test

array(['7 Coys of CRPF also comprising RAF PERSONNEL alongwith relief materiaal and 2 doctors alongwith medicines, health-kit reached Gujarat.',
       'can of olives , can of chickpeas , bag of pasta , bag of hard pretzels , ( also have sneakers I can donate )',
       'We are at the top of Fontamara 43 (area of Carrefour), at the top of the hill named label.. unrecognized characters. We did not get any assistance: water, food, tents, medicine.',
       ...,
       "The country's macroeconomic and fiscal stability has contributed to steady improvement in several economic and social indicators.",
       '* At the regular food aid coordination meeting in Dushanbe, chaired by WFP and attended by a broad representation of NGOs, international agencies and donors, the topic under discussion was how to achieve a better level of mutual understanding with local authorities, particularly in the fields of vulnerable group feeding and land lease projects, where vested interests can be an obstacle

In [210]:
doc2vec_features = doc2vec_trf.transform(X_test)
print(doc2vec_features.shape)
doc2vec_features

(6536, 100)


matrix([[ 7.1253721e-03,  1.4275378e-03, -1.6403912e-02, ...,
         -2.7752616e-03,  1.3460286e-03, -1.2717254e-02],
        [ 4.9492260e-03,  8.3419280e-03, -6.4557083e-03, ...,
          4.1161566e-03,  6.8386244e-03, -1.5606394e-02],
        [-2.3228582e-05,  2.2990115e-03, -9.7767143e-03, ...,
          1.6658346e-03,  4.2437436e-03, -1.7689072e-02],
        ...,
        [ 5.7200310e-03,  2.4893257e-04, -1.0689486e-02, ...,
          4.9673789e-03,  8.8404436e-03, -1.5657315e-02],
        [ 1.3653063e-03,  8.9902891e-04, -2.9417634e-02, ...,
         -2.2998024e-03,  6.2339050e-03, -2.4328450e-02],
        [ 4.3053534e-03,  5.2562519e-03, -1.7285153e-02, ...,
          3.5506324e-03,  5.7749161e-03, -1.8064328e-02]], dtype=float32)

Test Doc2Vec using a Pipeline

In [242]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())

              precision    recall  f1-score   support

   related-0       0.31      0.30      0.30      1532
   related-1       0.83      0.60      0.70      4950

   micro avg       0.68      0.53      0.59      6482
   macro avg       0.57      0.45      0.50      6482
weighted avg       0.70      0.53      0.60      6482

              precision    recall  f1-score   support

   request-0       0.89      0.68      0.77      5393
   request-1       0.28      0.59      0.38      1143

    accuracy                           0.67      6536
   macro avg       0.59      0.64      0.58      6536
weighted avg       0.78      0.67      0.70      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.95      0.97      6506
     offer-1       0.02      0.20      0.03        30

    accuracy                           0.94      6536
   macro avg       0.51      0.57      0.50      6536
weighted avg       0.99      0.94      0.97      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  refugees-0       0.98      0.76      0.86      6322
  refugees-1       0.06      0.43      0.10       214

    accuracy                           0.75      6536
   macro avg       0.52      0.60      0.48      6536
weighted avg       0.95      0.75      0.83      6536

              precision    recall  f1-score   support

     death-0       0.97      0.76      0.85      6244
     death-1       0.08      0.47      0.14       292

    accuracy                           0.75      6536
   macro avg       0.53      0.62      0.50      6536
weighted avg       0.93      0.75      0.82      6536

              precision    recall  f1-score   support

 other_aid-0       0.88      0.70      0.78      5660
 other_aid-1       0.17      0.41      0.24       876

    accuracy                           0.66      6536
   macro avg       0.53      0.55      0.51      6536
weighted avg       0.79      0.66      0.71      6536

                   

### 7. Extra Feature - Distance Between Message Word Vector to Categories Word Vectors 

We will use Glove as our space vectorizer as it is trained on a broad corpus and thus able to better describe texts / words

First, let's build the embeddings for the categories

In [213]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [214]:
categories_tokens = np.array([np.array(cat.split('_')) for cat in category_columns])

In [215]:
categories_tokens

array([array(['related'], dtype='<U7'), array(['request'], dtype='<U7'),
       array(['offer'], dtype='<U5'),
       array(['aid', 'related'], dtype='<U7'),
       array(['medical', 'help'], dtype='<U7'),
       array(['medical', 'products'], dtype='<U8'),
       array(['search', 'and', 'rescue'], dtype='<U6'),
       array(['security'], dtype='<U8'), array(['military'], dtype='<U8'),
       array(['child', 'alone'], dtype='<U5'),
       array(['water'], dtype='<U5'), array(['food'], dtype='<U4'),
       array(['shelter'], dtype='<U7'), array(['clothing'], dtype='<U8'),
       array(['money'], dtype='<U5'),
       array(['missing', 'people'], dtype='<U7'),
       array(['refugees'], dtype='<U8'), array(['death'], dtype='<U5'),
       array(['other', 'aid'], dtype='<U5'),
       array(['infrastructure', 'related'], dtype='<U14'),
       array(['transport'], dtype='<U9'),
       array(['buildings'], dtype='<U9'),
       array(['electricity'], dtype='<U11'),
       array(['tools'], dtype

In [216]:
glove_300d_w2v_mean = MeanEmbeddingTrainVectorizer(glove_300d_w2v)

In [217]:
glove_300d_w2v_mean.fit(categories_tokens, None)

In [218]:
cat_tokens_embeddings = glove_300d_w2v_mean.transform(categories_tokens)

In [219]:
print(cat_tokens_embeddings.shape)
print(cat_tokens_embeddings)

(36, 300)
[[ 0.12508     0.46811    -0.086999   ...  0.088037    0.17941999
  -0.27052   ]
 [-0.41626999 -0.49603    -0.22231001 ... -0.30825999 -0.29975
  -0.16089   ]
 [-0.19396    -0.2022      0.12097    ... -0.35918999 -0.19136
  -0.20799001]
 ...
 [-0.13273001 -0.21676999 -0.067073   ... -0.11713    -0.37046
   0.69020998]
 [-0.09252822 -0.14484501 -0.21590899 ... -0.13993001 -0.139598
   0.0952855 ]
 [ 0.0347955   0.0345      0.21976    ...  0.01706485  0.033145
  -0.1131545 ]]


In [220]:
cat_tokens_embeddings[0]

array([ 1.25080004e-01,  4.68109995e-01, -8.69989991e-02, -5.01410007e-01,
       -1.78369999e-01,  1.20580003e-01, -2.82119989e-01,  4.08140004e-01,
       -2.48469993e-01, -1.82319999e+00, -8.70039985e-02, -5.84609993e-02,
       -9.13850032e-03,  2.92499989e-01,  6.06190003e-02, -4.93750006e-01,
       -3.73620003e-01,  3.81080002e-01, -3.04690003e-01,  1.39229998e-01,
       -6.34540021e-01, -5.66239990e-02,  4.23099995e-02, -3.39699984e-02,
       -4.20120001e-01,  4.51339990e-01, -3.41100007e-01, -2.37819999e-01,
        5.82900010e-02,  3.24909985e-02,  1.67160004e-01,  6.21609986e-01,
       -4.91329993e-04, -3.72220017e-02, -3.15239996e-01, -3.86920005e-01,
        5.09299994e-01, -1.18060000e-01, -8.95330012e-02,  1.91430002e-01,
       -1.61880001e-01, -4.28250015e-01,  1.42199993e-01,  1.86289996e-01,
       -1.07410001e-02, -6.13229990e-01, -4.29800004e-01,  1.58490002e-01,
       -1.00910002e-02,  5.31729996e-01,  5.55189997e-02, -1.22680001e-01,
       -1.56979993e-01, -

In [221]:
test_msgs_embeddings = glove_300d_w2v_mean.transform(X_test)

In [222]:
print(test_msgs_embeddings.shape)
print(test_msgs_embeddings)

(6536, 300)
[[-2.49977112e-01  4.21655923e-02 -2.78507918e-01 ... -5.23655415e-01
  -1.04854256e-01  1.94041148e-01]
 [-2.29567811e-01  5.66051938e-02 -2.12730914e-01 ... -5.30342281e-01
  -1.78229362e-01  1.49230659e-01]
 [-2.79000521e-01 -4.36342700e-04 -2.78923422e-01 ... -5.11218488e-01
  -1.30129024e-01  1.15691535e-01]
 ...
 [-1.99770212e-01  1.73996594e-02 -2.41620466e-01 ... -5.53974032e-01
  -1.41777471e-01  2.10661530e-01]
 [-2.67834872e-01  2.57462841e-02 -2.99359620e-01 ... -5.63875675e-01
  -1.52957574e-01  1.38736591e-01]
 [-2.53114641e-01  4.64284085e-02 -2.42736846e-01 ... -5.61399400e-01
  -1.19366452e-01  1.51864842e-01]]


In [223]:
test_msgs_embeddings[0]

array([-2.49977112e-01,  4.21655923e-02, -2.78507918e-01, -6.29903749e-02,
       -5.44765353e-01,  1.21234590e-02, -5.07514961e-02,  2.06576541e-01,
       -2.96989977e-01, -1.18141806e+00,  1.06359333e-01,  4.30861302e-02,
       -3.79369855e-01,  1.08687595e-01, -5.44994995e-02, -9.19390693e-02,
       -2.88960010e-01,  3.22223455e-01,  1.92793339e-01, -4.70864177e-02,
       -1.30998254e-01, -1.19249672e-01, -3.86969373e-02,  7.96674415e-02,
       -9.18416753e-02,  1.17037833e-01,  1.41338661e-01,  1.13529168e-01,
        2.89718539e-01, -3.43774647e-01, -1.34874299e-01,  3.73800486e-01,
       -2.52926379e-01, -1.04804426e-01, -7.51641810e-01,  3.49596739e-02,
        2.70789742e-01, -3.88117105e-01, -2.06783757e-01,  1.99026003e-01,
       -9.24089327e-02,  2.86518354e-02, -4.40096319e-01,  5.09192467e-01,
       -2.46044412e-01, -2.41905540e-01,  2.62566298e-01, -9.45985243e-02,
        1.55663528e-02,  5.48544824e-02,  4.96799611e-02,  1.63628846e-01,
       -7.73277432e-02,  

In [225]:
from sklearn.metrics.pairwise import cosine_similarity

print(test_msgs_embeddings[:10].shape)
print(cat_tokens_embeddings.shape)
print(cosine_similarity(test_msgs_embeddings[:10],cat_tokens_embeddings).shape)

(10, 300)
(36, 300)
(10, 36)


#### Create a CategoriesSimilarity Transformer 

In [226]:
from sklearn.metrics.pairwise import cosine_similarity

class CategoriesSimilarity(object):
    
    def __init__(self, word2vec_model, categories_tokens):
        self.word2vec_model = word2vec_model
        self.categories_tokens = categories_tokens
        self.categories_vectors = None
        self.num_dims = word2vec_model.vector_size
        
    def compute_mean_embeddings(self, tokens_array):    
        mean_embeddings = np.empty([tokens_array.shape[0],self.num_dims])
        
        for i in range(tokens_array.shape[0]):
            doc_tokens = tokens_array[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings
                    
    def fit(self, X, y):
        self.categories_vectors = self.compute_mean_embeddings(self.categories_tokens)
        return self 

    def transform(self, X):
        mean_embeddings = self.compute_mean_embeddings(X)
        cats_similarities = cosine_similarity(mean_embeddings, self.categories_vectors)
            
        return cats_similarities

Test the transformer manually using test data

In [227]:
cats_sim_transf = CategoriesSimilarity(glove_300d_w2v, categories_tokens)

In [228]:
cats_sim_transf.fit(X_test, None)

In [229]:
cats_sim_transf.transform(X_test)

array([[0.25942149, 0.25508851, 0.28312271, ..., 0.19250859, 0.27899724,
        0.31365831],
       [0.26177502, 0.2623695 , 0.279754  , ..., 0.1884913 , 0.27933411,
        0.33824133],
       [0.2715689 , 0.27207593, 0.29562766, ..., 0.18817111, 0.29224368,
        0.33543405],
       ...,
       [0.24416702, 0.25761959, 0.27885234, ..., 0.18594135, 0.26677904,
        0.31532631],
       [0.24467236, 0.25305701, 0.26662035, ..., 0.16104647, 0.25017687,
        0.3029656 ],
       [0.24199337, 0.24450104, 0.25988893, ..., 0.16198157, 0.24640804,
        0.29208791]])

Test CategoriesSimilarity feature using a Pipeline

0

In [238]:
%%time

from sklearn.pipeline import FeatureUnion

cats_sim_pip = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, cat_tokens_embeddings)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

cats_sim_pip.fit(X_train, y_train)

y_pred = cats_sim_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("Score = ", cats_sim_pip.score(X_test, y_test))

# accuracy
print("Accuracy = ",accuracy_score(y_test,y_pred))
print("\n")

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[

              precision    recall  f1-score   support

   related-0       0.23      1.00      0.38      1532
   related-1       0.00      0.00      0.00      4950

   micro avg       0.23      0.24      0.24      6482
   macro avg       0.12      0.50      0.19      6482
weighted avg       0.06      0.24      0.09      6482

              precision    recall  f1-score   support

   request-0       0.83      1.00      0.90      5393
   request-1       0.00      0.00      0.00      1143

    accuracy                           0.83      6536
   macro avg       0.41      0.50      0.45      6536
weighted avg       0.68      0.83      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      1.00      1.00      6506
     offer-1       0.00      0.00      0.00        30

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.99      1.00      0.99      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

infrastructure_related-0       0.93      1.00      0.97      6111
infrastructure_related-1       0.00      0.00      0.00       425

                accuracy                           0.93      6536
               macro avg       0.47      0.50      0.48      6536
            weighted avg       0.87      0.93      0.90      6536

              precision    recall  f1-score   support

 transport-0       0.95      1.00      0.98      6219
 transport-1       0.00      0.00      0.00       317

    accuracy                           0.95      6536
   macro avg       0.48      0.50      0.49      6536
weighted avg       0.91      0.95      0.93      6536

              precision    recall  f1-score   support

 buildings-0       0.95      1.00      0.97      6209
 buildings-1       0.00      0.00      0.00       327

    accuracy                           0.95      6536
   macro avg       0.47      0.50      0.49      6536
we

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[

0.23439412484700123


ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

### 8. NLP Feature Selection using Sklearn Pipelines

#### 8.1 - All features together

In [232]:
%%time

from sklearn.pipeline import FeatureUnion

all_feats = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, cat_tokens_embeddings))
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

all_feats.fit(X_train, y_train)

y_pred = all_feats.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


              precision    recall  f1-score   support

   related-0       0.36      0.04      0.07      1532
   related-1       0.76      0.94      0.84      4950

   micro avg       0.75      0.73      0.74      6482
   macro avg       0.56      0.49      0.46      6482
weighted avg       0.67      0.73      0.66      6482

              precision    recall  f1-score   support

   request-0       0.83      0.95      0.89      5393
   request-1       0.25      0.08      0.12      1143

    accuracy                           0.80      6536
   macro avg       0.54      0.51      0.50      6536
weighted avg       0.73      0.80      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.99      0.99      6506
     offer-1       0.00      0.00      0.00        30

    accuracy                           0.99      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.99      0.99      0.99      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

missing_people-0       0.99      0.94      0.96      6462
missing_people-1       0.04      0.22      0.06        74

        accuracy                           0.93      6536
       macro avg       0.51      0.58      0.51      6536
    weighted avg       0.98      0.93      0.95      6536

              precision    recall  f1-score   support

  refugees-0       0.99      0.18      0.30      6322
  refugees-1       0.04      0.95      0.07       214

    accuracy                           0.20      6536
   macro avg       0.51      0.57      0.19      6536
weighted avg       0.96      0.20      0.29      6536

              precision    recall  f1-score   support

     death-0       0.99      0.15      0.26      6244
     death-1       0.05      0.95      0.09       292

    accuracy                           0.19      6536
   macro avg       0.52      0.55      0.18      6536
weighted avg       0.94      0.19      0.25      6

In [233]:
score = all_feats.score(X_test, y_test)

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [234]:
score

0.005660954712362301

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize, ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
X_train.shape

(19662,)

In [14]:
X_train

array(['It works as a fertilizer enhancement, a composting additive, and even as an alternative to household cleaning products, according to EMRO.',
       'I need a job to feed my family. I am an electrician and a musician. I perform harmonization and orchestration in Mexico. I speak English and French. Let me know how I can help',
       'The report also warns that if the clan fighting extends to sorghum-producing areas in the south during this critical stage of crop harvest, it could further hamper harvesting and exert a negative impact on food security throughout Somalia.',
       ...,
       'HELP THE EARTHQUAKE VICTIMS IN HAITI http tinyurl.com yk3bspe links to many resources where you can make donations. Broke? donate time',
       'The low snowfall pattern and the snowfall differentiation among catchments are clearly reflected in the irrigated wheat production levels.',
       "Having fun at #Lexi 's ! Real food in real house with electrical lighting . Good times . I feel human

In [15]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

ValueError: WRITEBACKIFCOPY base is read-only

In [ ]:
print(X_test.shape)

In [ ]:
print(y_pred.shape)
y_pred

In [ ]:
y_pred[:,0]

In [ ]:
y_test

In [ ]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'multi_clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'multi_clf__estimator__bootstrap', 'multi_clf__estimator__class_weight', 'multi_clf__estimator__criterion', 'multi_clf__estimator__max_depth', 'multi_clf__estimator__max_features', 'multi_clf__estimator__max_leaf_nodes', 'multi_clf__estimator__min_impurity_decrease', 'multi_clf__estimator__min_impurity_split', 'multi_clf__estimator__min_samples_leaf', 'multi_clf__estimator__min_samples_split', 'multi_clf__estimator__min_weight_fraction_leaf', 'multi_clf__estimator__n_estimators', 'multi_clf__estimator__n_jobs', 'multi_clf__estimator

In [ ]:
parameters = {
        'multi_clf__estimator__n_estimators': [20,50],
        'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................
[CV] ...................... , score=0.22017088800732376, total= 2.7min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV] ...................... , score=0.22978333841928594, total= 2.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s


[CV] ...................... , score=0.22642660970399756, total= 2.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min finished


In [32]:
cv.best_params_

{}

In [33]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

             precision    recall  f1-score   support

  related-0       0.55      0.32      0.41      1565
  related-1       0.81      0.91      0.86      4941

avg / total       0.74      0.77      0.75      6506

             precision    recall  f1-score   support

  request-0       0.89      0.98      0.93      5420
  request-1       0.81      0.39      0.53      1134

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6524
    offer-1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

               precision    recall  f1-score   support

aid_related-0       0.71      0.89      0.79      3900
aid_related-1       0.75      0.46      0.57      2654

  avg / total       0.73      0.72      0.70      6554

                precision    recall  f1-score   support

medical_help-0       0.93      1.00      0.96      6038
medical_help-1     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [122]:
# Output a pickle file for the model
joblib.dump(cv, 'classifier.pkl') 

['initial_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.